# Sprint 10 — Churn Prediction (Beta Bank)

**Autora:** Diana Marlene Reyes Fraire  
**Objetivo:** Predecir abandono de clientes maximizando **F1** (meta ≥ 0.59) y reportar **AUC-ROC**.  
**Datos:** `data/Churn.csv` (local, ignorado por Git)

## Índice
1. Cargar datos y revisión inicial
2. Limpieza y preparación de características
3. División train/valid/test (estratificada)
4. Baseline y modelo **sin** tratar el desbalance
5. Mejora con dos enfoques de desbalance (weights y sobre/submuestreo)
6. Ajuste de umbral y métricas (F1, ROC, AUC-ROC)
7. Prueba final y conclusiones


## Importación de librerías

In [25]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, roc_auc_score, classification_report, confusion_matrix, RocCurveDisplay

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as ImbPipeline

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 120)
RANDOM_STATE = 42

## Carga de los datos

In [26]:
# Cargar el Dataset local
df = pd.read_csv("../data/Churn.csv")

## Exploración inicial del dataset

En esta sección se cargan los datos del banco **Beta Bank**, correspondientes a clientes y su estado de permanencia o abandono.  
El objetivo es revisar su estructura, dimensiones, tipos de datos y valores faltantes y duplicados antes del preprocesamiento.

In [27]:
# Vista previa de los datos
display(df.head())


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [28]:
# Imprime la información general sobre el DataFrame "df"
print('Información general del DataFrame:')
print()
df.info()

Información general del DataFrame:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [29]:
# Revisión de duplicados
print("\nDuplicados encontrados:", df.duplicated().sum())


Duplicados encontrados: 0


In [30]:
# Revisión de valores nulos
print("Valores nulos por columna:")
df.isna().sum()


Valores nulos por columna:


RowNumber            0
CustomerId           0
Surname              0
CreditScore          0
Geography            0
Gender               0
Age                  0
Tenure             909
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
dtype: int64

In [31]:
# Estadísticas descriptivas de las variables numéricas
print('Estadísticas descriptivas:')
df.describe()

Estadísticas descriptivas:


,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,9091.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,4.997690,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.894723,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


### Resumen de exploración inicial

- El dataset contiene **10,000 registros** y **14 columnas**.
- La variable objetivo es **`Exited`**:  
  - `1` = el cliente **abandonó** el banco.  
  - `0` = el cliente **permanece**.
- Las columnas **`RowNumber`**, **`CustomerId`** y **`Surname`** son identificadores que **no aportan valor predictivo**, por lo que se eliminarán durante el preprocesamiento.
- La variable `Exited` presenta un **desequilibrio de clases**:  
  aproximadamente **20%** de los clientes se fueron y **80%** permanecieron.  
  Este desbalance deberá corregirse más adelante mediante técnicas como **ponderación de clases** o **sobremuestreo**.
- La columna `Tenure` tiene valores faltantes (~9% de los registros).  
  Dado que representa años de permanencia y que la variable es importante, lo mejor será eliminar estas filas ya que esto no afectará demasiado el tamaño del dataset y evitará introducir sesgos en el análisis o imputaciones artificiales.
- Se estandarizarán los nombres de las columnas a **minúsculas** para asegurar consistencia y evitar errores durante la codificación y el modelado.
- Los datos en la variable `Tenure` probablemente sean del tipo `float` debido a los valores nulos, por lo que se convertirá a tipo **entero (`int`)** tras eliminar los valores nulos para reflejar correctamente los años completados de permanencia.
- No se encontraron registros duplicados.


## Preparación de características

En esta sección se realiza la preparación de los datos para el modelado.  
Las acciones incluyen:
- Eliminación de columnas irrelevantes (identificadoras).  
- Normalización de nombres de columnas.  
- Manejo de valores faltantes.  
- Ajuste de tipos de datos.  
- Separación de características numéricas y categóricas.  
- Codificación y escalado de variables.

Estas transformaciones son esenciales para que los modelos de *Machine Learning* trabajen con datos consistentes y comparables.

In [ ]:
# Copia del DataFrame original
df_clean = df.copy()

# 1. Normalizar nombres de columnas
df_clean.columns = df_clean.columns.str.strip().str.lower()

# 2. Eliminar columnas irrelevantes
df_clean = df_clean.drop(columns=['rownumber', 'customerid', 'surname'])

# 3. Revisar valores nulos
print("Valores nulos antes de limpiar:")
print(df_clean.isna().sum())

# 4. Eliminar filas con nulos en tenure (~9%)
df_clean = df_clean.dropna(subset=['tenure'])

# 5. Convertir tenure a entero
df_clean['tenure'] = df_clean['tenure'].astype(int)

# Verificación final
print("\nValores nulos después de limpiar:")
print(df_clean.isna().sum())
print("\nTipos de datos:")
display(df_clean.dtypes)
display(df_clean.head())


Valores nulos antes de limpiar:
creditscore          0
geography            0
gender               0
age                  0
tenure             909
balance              0
numofproducts        0
hascrcard            0
isactivemember       0
estimatedsalary      0
exited               0
dtype: int64

Valores nulos después de limpiar:
creditscore        0
geography          0
gender             0
age                0
tenure             0
balance            0
numofproducts      0
hascrcard          0
isactivemember     0
estimatedsalary    0
exited             0
dtype: int64

Tipos de datos:


creditscore          int64
geography           object
gender              object
age                  int64
tenure               int64
balance            float64
numofproducts        int64
hascrcard            int64
isactivemember       int64
estimatedsalary    float64
exited               int64
dtype: object

,creditscore,geography,gender,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [33]:
# Separar variable objetivo y características
target = 'exited'
X = df_clean.drop(columns=[target])
y = df_clean[target]

print("Tamaño de X:", X.shape)
print("Tamaño de y:", y.shape)

Tamaño de X: (9091, 10)
Tamaño de y: (9091,)


In [34]:
# 70% train, 15% valid, 15% test
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

X_valid, X_test, y_valid, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
)

print(f"Tamaños:\nTrain: {len(X_train)}\nValid: {len(X_valid)}\nTest: {len(X_test)}")
print("\nDistribución de clases (train):")
print(y_train.value_counts(normalize=True))

Tamaños:
Train: 6363
Valid: 1364
Test: 1364

Distribución de clases (train):
exited
0    0.796008
1    0.203992
Name: proportion, dtype: float64


In [35]:
# Identificar columnas numéricas y categóricas
num_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_cols = X.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()

print("Numéricas:", num_cols)
print("Categóricas:", cat_cols)

# Preprocesador
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore', drop='first'), cat_cols)
    ]
)

# Prueba rápida de transformación
X_train_prepared = preprocessor.fit_transform(X_train)
X_valid_prepared = preprocessor.transform(X_valid)

print("\nTamaño después del pre-procesamiento:", X_train_prepared.shape)

Numéricas: ['creditscore', 'age', 'tenure', 'balance', 'numofproducts', 'hascrcard', 'isactivemember', 'estimatedsalary']
Categóricas: ['geography', 'gender']

Tamaño después del pre-procesamiento: (6363, 11)


### Resumen de preparación de características

- Se eliminaron las columnas `rownumber`, `customerid` y `surname` por no aportar valor predictivo.  
- Se estandarizaron los nombres de las columnas a minúsculas.  
- Se eliminaron las filas con valores nulos en `tenure` (~9%).  
- Se convirtió `tenure` a tipo entero (`int`).  
- Se dividió el dataset en **train (70 %)**, **valid (15 %)** y **test (15 %)** para ofrecer al modelo una mayor cantidad de ejemplos de la clase minoritaria (clientes que abandonan el banco).
- Se aplicó **estratificación** en la división de datos para mantener la proporción original de clases (≈ 80 % permanecen, 20 % se van) en todos los subconjuntos.
- Las variables categóricas se codificarán mediante **One-Hot Encoding** para convertir texto en valores numéricos.
- Las variables numéricas se escalarán con **StandardScaler**, que las transforma a media 0 y desviación estándar 1, evitando que magnitudes diferentes influyan de forma desigual en el entrenamiento.